## Cadences Network Visualization with Louvain Communities



In [50]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


# B.  Select Corpus

* The **CorpusBase** class is a convenient way to find patterns in any given list of pieces.
* The pieces are provided as a **list**, within square brackets and separated by commas.  
* The bracketed list is then contained within the parentheses of `CorpusBase()`
* For example: `corpus CorpusBase(
       ['https://crimproject.org/mei/CRIM_Mass_0006_1.mei',
       'https://crimproject.org/mei/CRIM_Mass_0006_2.mei',
       'https://crimproject.org/mei/CRIM_Mass_0006_3.mei'])`
       
* Read the documentation:  `print(CorpusBase.batch.__doc__)`


In [68]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0019.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0019_1.mei',
                    'https://crimproject.org/mei/CRIM_Mass_0019_2.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0019_3.mei'
                    ,'https://crimproject.org/mei/CRIM_Mass_0019_4.mei',
                    'https://crimproject.org/mei/CRIM_Mass_0019_5.mei'])

Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.


In [44]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0019.mei'])

Memoized piece detected.


In [69]:


list_of_dfs = corpus.batch(ImportedPiece.cadences, metadata=True)
combined_df = pd.concat(list_of_dfs, ignore_index=True)
combined_df.dropna(inplace=True)
combined_df['all_features'] = combined_df['CadType'] + "_" + combined_df['Tone'] + "_" + combined_df['Evaded'].apply(str) + "_" + combined_df['Beat'].apply(str)
combined_df['some_features'] = combined_df['CadType'] + "_" + combined_df['Tone']
combined_df.copy
combined_df["previous_all_feat"] = combined_df.groupby("Title")['all_features'].shift(1)
combined_df

,CadType,Evaded,LeadingTones,Low,RelLow,Tone,RelTone,TSig,Measure,Beat,Progress,SinceLast,ToNext,Composer,Title,all_features,some_features,previous_all_feat
0,Clausula Vera,False,1.0,G3,P1,G,P1,4/2,7,1.0,0.090909,48.0,36.0,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,Clausula Vera_G_False_1.0,Clausula Vera_G,NaN
1,Clausula Vera,True,1.0,G3,P1,G,P1,4/2,11,3.0,0.159091,36.0,52.0,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,Clausula Vera_G_True_3.0,Clausula Vera_G,Clausula Vera_G_False_1.0
2,Authentic,True,1.0,B3,M3,G,P1,4/2,18,1.0,0.257576,52.0,16.0,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,Authentic_G_True_1.0,Authentic_G,Clausula Vera_G_True_3.0
3,Authentic,False,1.0,G3,P1,G,P1,4/2,20,1.0,0.287879,16.0,72.0,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,Authentic_G_False_1.0,Authentic_G,Authentic_G_True_1.0
4,Clausula Vera,True,1.0,F3,-M2,C,-P5,4/2,29,1.0,0.424242,72.0,28.0,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,Clausula Vera_C_True_1.0,Clausula Vera_C,Authentic_G_False_1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,Clausula Vera,False,0.0,F3,-M2,A,M2,4/2,60,3.0,0.852113,12.0,12.0,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,Clausula Vera_A_False_3.0,Clausula Vera_A,Authentic_A_False_3.0
146,Clausula Vera,False,0.0,D3,-P4,D,-P4,4/2,62,3.0,0.880282,2.0,24.0,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,Clausula Vera_D_False_3.0,Clausula Vera_D,Clausula Vera_A_False_3.0
147,Clausula Vera,False,0.0,D3,-P4,D,-P4,4/2,65,3.0,0.922535,24.0,8.0,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,Clausula Vera_D_False_3.0,Clausula Vera_D,Clausula Vera_D_False_3.0
148,Authentic,False,0.0,D3,-P4,D,-P4,4/2,66,3.0,0.936620,8.0,12.0,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,Authentic_D_False_3.0,Authentic_D,Clausula Vera_D_False_3.0


### Network Visualization with Louvain Communities

* We first create a list of **successive cadences** in each piece.  These are **Cadence Pairs**.
* Each item in each pair records the following features:  **type**, **tone**, **evaded status**, and **beat strength of last note in the cadence**
* The pairs are thus highly distinct--consisting of four dimensions of two cadences each, such as: `('Clausula Vera_G_False_1.0', 'Clausula Vera_G_True_3.0')`
* This will create a varied network.

In [70]:
combined_df = combined_df.dropna(subset=['previous_all_feat', 'all_features'])
cadence_pairs = list(zip(combined_df.previous_all_feat, combined_df.all_features))
cadence_pairs

[('Clausula Vera_G_False_1.0', 'Clausula Vera_G_True_3.0'),
 ('Clausula Vera_G_True_3.0', 'Authentic_G_True_1.0'),
 ('Authentic_G_True_1.0', 'Authentic_G_False_1.0'),
 ('Authentic_G_False_1.0', 'Clausula Vera_C_True_1.0'),
 ('Clausula Vera_C_True_1.0', 'Clausula Vera_C_True_3.0'),
 ('Clausula Vera_C_True_3.0', 'Authentic_C_False_1.0'),
 ('Authentic_C_False_1.0', 'Clausula Vera_A_True_1.0'),
 ('Clausula Vera_A_True_1.0', 'Altizans Only_A_False_1.0'),
 ('Altizans Only_A_False_1.0', 'Clausula Vera_D_False_1.0'),
 ('Clausula Vera_D_False_1.0', 'Clausula Vera_A_False_3.0'),
 ('Clausula Vera_A_False_3.0', 'Authentic_D_False_1.0'),
 ('Authentic_D_False_1.0', 'Altizans Only_D_False_1.0'),
 ('Altizans Only_D_False_1.0', 'Authentic_G_False_1.0'),
 ('Clausula Vera_G_False_1.0', 'Clausula Vera_G_False_1.0'),
 ('Clausula Vera_G_False_1.0', 'Clausula Vera_D_False_3.0'),
 ('Clausula Vera_D_False_3.0', 'Authentic_G_False_1.0'),
 ('Authentic_G_False_1.0', 'Clausula Vera_G_True_3.0'),
 ('Clausula Vera_G

In [71]:
combined_df['all_features'].nunique()

36

### A Directed Graph of Pairs

* Directed graphs reflect the succession of events, and create a centrum of the most common 'goal' cadence.  Less frequent successions are at the edge of the graph

In [72]:
def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G
pyvis_graph = Network(notebook=False, width="1800", height="1400", bgcolor="white", font_color="black", directed=True)

G = nx.DiGraph()
# G.add_edges_from(unique_pairs)
G.add_edges_from(cadence_pairs)
# G = add_communities(G)
pyvis_graph.from_nx(G)
pyvis_graph.show('Mass_19_cad_graph.html')